# W205 Project 3
## AGM Business Improvement

### Team Members
#### Siddhartha Gupte
#### Kumar Kallurupalli
#### Tigran Poladian

Data science demonstration for the AGM data science team using the BART transportation system.

### BART TRACK LAYOUT

![Bart Map](../data/bart_map.png)

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import neo4j
import csv
import math
import numpy as np
import pandas as pd

import psycopg2

import pymongo

import json

import geopy
from geopy.geocoders import Nominatim

from matplotlib import pyplot as plt

### Support Functions
Reused from W205 Code / Labs

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

cursor = connection.cursor()

In [4]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

### Drop tables, create and load from CSV files

In [6]:
connection.rollback()

query = """

drop table if exists stations;
"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [7]:
connection.rollback()

query = """

drop table if exists lines;

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [8]:
connection.rollback()

query = """

drop table if exists travel_times;

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [9]:
connection.rollback()

query = """

create table stations (
    station varchar(32),
    latitude numeric(9,6),
    longitude numeric(9,6),
    transfer_time numeric(3),
    primary key (station)
);

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [10]:
connection.rollback()

query = """

create table lines (
    line varchar(6),
    sequence numeric(2),
    station varchar(32),
    primary key (line, sequence)
);

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [11]:
connection.rollback()

query = """

create table travel_times (
    station_1 varchar(32),
    station_2 varchar(32),
    travel_time numeric(3),
    primary key (station_1, station_2)
);

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [12]:
# NOTE: use location of data files
connection.rollback()

query = """

copy stations
from '/user/projects/w205_project3/data/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [13]:
connection.rollback()

query = """

copy lines
from '/user/projects/w205_project3/data/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

In [14]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/w205_project3/data/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor = connection.cursor()
cursor.execute(query)

connection.commit()

### Establish Neo4j connection and helper functions

In [17]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))
session = driver.session(database="neo4j")

In [18]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)
  
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)

def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)


In [19]:
my_neo4j_wipe_out_database()

### Create graphs from csv station files

#### Query the list of stations and create the departure and arrival nodes in the graph

Use the query from 3.2.1 "Query the list of stations"

For each station X, create two nodes:
* depart X
* arrive X

Use the function my_neo4j_create_node() defined above

For example, West Oakland:
* my_neo4j_create_node('depart West Oakland')
* my_neo4j_create_node('arrive West Oakland')

In [ ]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)

#### Query the list of stations and the lines they serve, create line nodes, and create relationships between the line nodes and the departure and arrival nodes with weight 0

Use the query from "Query the list of stations and the lines they serve"

For each station X and each line Y that the station serves:
* Create a line node
* Create a relationship from the departure node to the line node with weight 0
* Create a relationship from the line node to the arrival node with weight 0

Use the function my_neo4j_create_relationship_one_way() defined above to create the relationships

In [ ]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)

#### Query the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight

Use the query from 3.2.5 "Query the list of all possible line transfers and the transfer times"

For each station X, from line Y, to line Z, create a relationship from Y's line node to Z's line node with the weight set to the transfer time

In [ ]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)

#### Query the list of all segments between each station and its adjoining stations, create a relationship for each segment both ways

Use the query from 3.2.7 "Query the list of all segments between each station and its adjoining stations"

For each segment from station X to station Y on line Z, create two relationships:
* From X's line node to Y's line node with travel time
* From Y's line node to X's line node with travel time

Use the function my_neo4j_create_relationship_two_way() defined above which will create both relationships 

In [ ]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)

### Begin PROJECT 3 CODE HERE

#### Define search algorithms

In [ ]:
# ************************************************************************************************************
# Shortest Path algorithm from W205 labs - uses dijkstra
# ************************************************************************************************************
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1

In [ ]:
# ***************************************************************************
# UNIT TEST dijkstra shortest path
# ***************************************************************************
my_neo4j_shortest_path("depart Ashby","arrive West Oakland")

In [ ]:
# ***************************************************************************
# Minimum Spanning Tree
# ***************************************************************************
def my_neo4j_mst(station):
    "wipe out mst relationships"
    
    query = "match (node)-[relationship:MST]->() delete relationship"
    session.run(query)
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)
    
    query = """

    MATCH (n:Station {name: $source})
    CALL gds.alpha.spanningTree.minimum.write('ds_graph',
                                          {startNodeId: id(n),
                                           relationshipWeightProperty: 'weight',
                                           writeProperty: 'MST',
                                           weightWriteProperty: 'writeCost'
                                          }
                                         )
    YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
    RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;
    """
    session.run(query,source=station)
    
    query = """

    MATCH path = (n:Station {name: $source})-[:MST*]-()
    WITH relationships(path) AS rels
    UNWIND rels AS rel
    WITH DISTINCT rel AS rel
    RETURN startNode(rel).name AS source, endNode(rel).name AS destination, rel.writeCost AS cost

    """
    
    session.run(query,source=station)

In [ ]:
# ******************************************************************************************
# UNIT TEST Minimum Spanning Tree
# ******************************************************************************************
my_neo4j_mst("depart Ashby")

In [ ]:
# ******************************************************************************************
# ADD ADDITIONAL neo4j gds algorithms and use directly in the business scenarios
# ******************************************************************************************

#### Project To Do
1. Add modules to injest real time traffic data (use Redis or Redis fixed document as a proxy)
2. Import train schedules, planned route outages (use MongoDB)
3. Based on 1 and 2 create new graphs or modify cost / weights across nodes on existing graphs to eliminate paths and change time cost across nodes
4. Create 3 or more search, sort algorithms. Use the above as a template
5. Overlay Google maps API


#### Business Scenario 1

Select customers from the customers table who are within the BART transportation region

Run EDA using neo4j
   * Find clusters of customers, where concentrated most
   * Find nearest and furthest BART station to the clusters
   * Recommend pop up stores and new permanent locations

In [ ]:
# ***********************************************************************************************************************
# Business Scenario 1
#
# Pseudocode:
#
# 1. Get list of unique zip codes associated to the set of BART stations using lat lon of station locations
#   -- lat lon from station table
# 2. From the customers table, find all customers with zip codes in the list of zip codes
# 3. Prepare histogram of customer counts per zip code
# 4. Create neo4j graph of nodes for zip codes along with graph of stations 
#   -- connect zip code nodes to station nodes with links indicating distance so distance from zip to station within 
#      same zip should be 0
#   -- refine further ... use customer address (and google maps api?) to compute actual distance to each BART station
# 5. Can we use any of the centrality type measures for this scenario?
# 6. refine further ... use customer address (and google maps api?) to compute actual distance to each BART station
#
# Notes:
#   Reverse zip code locator code from: 
#   https://www.geeksforgeeks.org/find-the-location-with-specified-latitude-and-longitude-using-python/
#   https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/?ref=rp
# **********************************************************************************************************************
def neo4j_create_node(station_name,zip_code="",tt=0,pop=0):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name, zip: $zip_code, pop: $pop, transfer_time: $tt})
    
    """
    
    session.run(query, station_name=station_name,zip_code=zip_code,pop=pop,tt=tt)
    
def find_customer_count(zipcode):
    connection.rollback()

    rollback_before_flag = True
    rollback_after_flag = True

    query = """ 
    select zip, count(*)
    from customers
    where zip = '%s'
    group by zip
    order by zip
    """.replace("%s",zipcode)
    
    df_count = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
    
    return df_count['count']


def neo4j_create_station():
    connection.rollback()

    query = """

    select station, latitude, longitude, transfer_time
    from stations
    order by station

    """

    cursor.execute(query)

    connection.rollback()

    rows = cursor.fetchall()

    for row in rows:
    
        station = row[0]
        lat = row[1]
        lon = row[2]
        tt = row[3]
        
        # Find Zip code
        location = geolocator.reverse(str(lat)+","+str(lon))
        address = location.raw['address']
        zipcode = address.get('postcode')
        
        # Find number of customers
        #if zipcode == None:
        #    pop = 0
        #else:
        #    pop = find_customer_count(zipcode)
        
        #neo4j_create_node(station,zipcode,str(tt),str(pop))
        neo4j_create_node(station,zipcode,str(tt),0)

def neo4j_create_zip_node(node_name, attr):
    "create a node with label Station"
    
    #query = "\"\" CREATE (:"+str(attr)+" {name: " + str(node_name) + "}) \"\" "
    
    query = """
    CREATE (:Zip {name: $name})
    """
    
    session.run(query,name=node_name)

def neo4j_create_relationship_one_way(from_node, to_node, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Zip), 
          (to:Zip)
    WHERE from.name = $from_node and to.name = $to_node
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_node=from_node, to_node=to_node, weight=weight)

def get_station_name_by_zip(zipcode):
    query = """
    
    MATCH(n:Station {zip: $zipcode}) 
    RETURN n.name
    """
    
    res = session.run(query, zipcode=zipcode)
        
    return([record["n.name"] for record in res])

def neo4j_add_pop_attribute(node_name, pop_value):
    
    #print(node_name[0])
    #query = """
    
    #MATCH(n:Station {name: $node_name}) 
    #SET n.pop = $pop_value
    #"""

    #session.run(query, node_name=node_name, pop_value=pop_value)
    
    query = """
    
    MATCH(n:Station {name: '%s'}) 
    SET n.pop = '%t'
    """.replace("%s",str(node_name[0])).replace("%t",str(pop_value))
    
    #print(query)
    session.run(query)

def neo4j_find_travel_time(station1, station2):
    print(str(station1) + "," + str(station2))
    connection.rollback()

    rollback_before_flag = True
    rollback_after_flag = True

    query = """ 
    select travel_time
    from travel_times
    where (station_1 = '%s1' and station_2 = '%s2') or (station_1 = '%s2' and station_2 = '%s1')
    """.replace("%s1",station1).replace("%s2",station2)
    
    df_tt = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
    
    print(df_tt['travel_time'][0])
    return (df_tt['travel_time'][0])

In [ ]:
# *********************************************************
# UNIT TEST
# *********************************************************
neo4j_find_travel_time("MacArthur", "Ashby")

In [ ]:
# ***********************************************************************************************************************
# 1. Get list of unique zip codes associated to the set of BART stations using lat lon of station locations
#   -- lat lon from station table
# ***********************************************************************************************************************

connection.rollback()

query = """

select latitude, longitude, station
from stations

"""
cursor.execute(query)
connection.rollback()
rows = cursor.fetchall()

zips = []

for i, row in enumerate(rows):
    location = geolocator.reverse(str(row[0])+","+str(row[1]))
    address = location.raw['address']
    #city = address.get('city', '')
    #state = address.get('state', '')
    #country = address.get('country', '')
    #code = address.get('country_code')
    zipcode = address.get('postcode')
    #print(i)
    #print('City : ',city)
    #print('State : ',state)
    #print('Country : ',country)
    #print('Zip Code : ', zipcode)
    
    if zipcode == None:
      continue
    zips.append(zipcode)
    
zips_unique = set(zips)
print(zips_unique)

In [ ]:
# ***********************************************************************************************************************
# 2. From the customers table, find all customers with zip codes in the list of zip codes
# ***********************************************************************************************************************
#print(zips_unique)
#print(zips)

zip_list = ''
for z in zips_unique:
    zip_list += '\'' + str(z) + '\'' + ','

zip_list = zip_list[:-1] # remove extra last comma

rollback_before_flag = True
rollback_after_flag = True

query = """ 
select zip, count(*)
from customers
where zip in (%s)
group by zip
order by zip
""".replace("%s",zip_list)

df_customers = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df_customers

In [ ]:
# ***********************************************************************************************************************
# 3. Prepare barplot of customer counts per zip code
# **********************************************************************************************************************

df_customers.set_index(df_customers['zip'],inplace=True)
df_customers.plot(kind='bar',figsize=(12,6),title='AGM Customers per Zip Code').grid(axis='y')
plt.xlabel('Zip Codes')
plt.ylabel('Number of Customers')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# **********************************************************************************************************************
# 4a. Create neo4j graph of nodes for zip codes along with graph of stations 
#   -- connect zip code nodes to station nodes with links indicating distance so distance from zip to station within 
#      same zip should be 0
#   
# **********************************************************************************************************************
my_neo4j_wipe_out_database()
neo4j_create_station()

In [ ]:
# **********************************************************************************************************************
# 4b. Add zip code to each station node based on station name 
# **********************************************************************************************************************
for i,z in enumerate(df_customers['zip']):
    #neo4j_create_node(z,"Count")
    #neo4j_create_relationship_one_way(z, z, str(df_customers['count'][i]))
   
    node_name = get_station_name_by_zip(z)
    pop_value = str(df_customers['count'][i])
    
    #print(pop_value)
    neo4j_add_pop_attribute(node_name, pop_value)


In [ ]:
def neo4j_create_relationship(from_node, to_node, color, weight):
    "create a relationship between two stations with a weight:travel time"
    
    if color == 'blue':
        query = """
    
        MATCH (from:Station), 
          (to:Station)
        WHERE from.name = $from_node and to.name = $to_node
        CREATE (to)-[:BLUE {weight: $weight}]->(from)-[:BLUE {weight: $weight}]->(to)
    
        """
    elif color == 'gray':
        query = """
    
        MATCH (from:Station), 
          (to:Station)
        WHERE from.name = $from_node and to.name = $to_node
        CREATE (to)-[:GRAY {weight: $weight}]->(from)-[:GRAY {weight: $weight}]->(to)
    
        """
    elif color == 'green':
        query = """
    
        MATCH (from:Station), 
          (to:Station)
        WHERE from.name = $from_node and to.name = $to_node
        CREATE (to)-[:GREEN {weight: $weight}]->(from)-[:GREEN {weight: $weight}]->(to)
    
        """
    
    elif color == 'orange':
        query = """
    
        MATCH (from:Station), 
          (to:Station)
        WHERE from.name = $from_node and to.name = $to_node
        CREATE (to)-[:ORANGE {weight: $weight}]->(from)-[:ORANGE {weight: $weight}]->(to)
    
        """
    
    elif color == 'red':
        query = """
    
        MATCH (from:Station), 
          (to:Station)
        WHERE from.name = $from_node and to.name = $to_node
        CREATE (to)-[:RED {weight: $weight}]->(from)-[:RED {weight: $weight}]->(to)
    
        """
    
    elif color == 'yellow':
        query = """
    
        MATCH (from:Station), 
          (to:Station)
        WHERE from.name = $from_node and to.name = $to_node
        CREATE (to)-[:YELLOW {weight: $weight}]->(from)-[:YELLOW {weight: $weight}]->(to)
    
        """
    
    else:
        return
    
    session.run(query, from_node=from_node, to_node=to_node, weight=weight)

In [ ]:
# *****************************************
# UNIT TEST
# *****************************************
neo4j_create_relationship("Ashby", "MacArthur", "red", 80)

In [ ]:
# **********************************************************************************************************************
# 4c. Link stations, add travel time and line color attribute to links
# **********************************************************************************************************************
line_colors = ['blue','gray','green','orange','red','yellow']

connection.rollback()

rollback_before_flag = True
rollback_after_flag = True

for lc in line_colors:
    query = """ 
    select sequence, station
    from lines
    where line = '%s'
    """.replace("%s",lc)
    
    df_line = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
    
    cnt = len(df_line['station'])
    
    if cnt > 1:
        for i in range (cnt-1):
            #print(df_line['station'][i])
            #print(df_line['station'][i+1])
            stn1 = df_line['station'][i]
            stn2 = df_line['station'][i+1]
            
            #print(str(stn1) + "," + str(stn2))
            tt = neo4j_find_travel_time(stn1, stn2)
            #print(tt)
            neo4j_create_relationship(stn1,stn2,lc,str(tt))
    else:
        continue
        

#### Business Scenario 2

Given traffic congestion "real time" data, compute shortest BART transportation path to avoid congestion

* Import traffic data either from Redis or canned file
* Update weights on links to make congested paths more costly
* Run shortest path (or other) algorithms
   
 (Traffic congestion data -- extreme, high, medium, low as input feed. Update transit times by
 factors of 100, 10, 2, 1. Run route planning algorithm based on congestion.)


In [ ]:
# **********************************************************************************************************************
# Business Scenario 2
#
# Given traffic congestion "real time" data, compute shortest 
# BART transportation path to avoid congestion
#
# Import traffic data either from Redis or canned file
# Update weights on links to make congested paths more costly
# Run shortest path (or other) algorithms
#   
# (Traffic congestion data -- extreme, high, medium, low as input feed. Update transit times by
# factors of 100, 10, 2, 1. Run route planning algorithm based on congestion.)
# *********************************************************************************************************************

In [ ]:
# ADD SCENARIO 2

#### Business Scenario 3
Given scheduled station closures and track segment out of service, re-evaluate best locations for popup locations or perhaps deploy delivery robots to pick up from those locations not accessible to customers and bring products to open locations. For example, if Oakland marathon is running with street closures, customers cannot access some areas of the city.

* Import schedule file (from Redis, MongoDB or canned CSV file)
* Use file remove stations from neo4j however do not remove associated pickup location thereby causing a dangling node without a link.
* Deploy mobile robots to dangling nodes and deliver product to nearest open pickup location
* Compute next nearest location also accounting for removed Links due to planned track maintenance

In [ ]:
# *********************************************************************************************************************
# Business Scenario 3
#
# Given scheduled station closures and track segment out of service,  
# re-evaluate best locations for popup locations or perhaps deploy
# delivery robots to pick up from those locations not accessible to
# customers and bring products to open locations.  For example, if
# Oakland marathon is running with street closures, customers cannot 
# access some areas of the city.
#
# Import schedule file (from Redis, MongoDB or canned CSV file)
# Use file remove stations from neo4j however do not remove associated pickup location 
#   thereby causing a dangling node without a link.
# Deploy mobile robots to dangling nodes and deliver product to nearest open pickup location
# Compute next nearest location also accounting for removed Links due to planned
#   track maintenance
#   
# ********************************************************************************************************************

In [ ]:
# ADD SCENARIO 3